##Primer Dataframe: Tasas  

Pegar en la carpeta date_minutes.pkl y 1M Rate.csv

In [ ]:
!pip install FedTools
import pandas_datareader.data as web
import datetime
import pandas as pd
from FedTools import MonetaryPolicyCommittee
from FedTools import FederalReserveMins
from google.colab import drive, files


In [ ]:
path_minutes_published='/content/date_minutes.pkl'
df_fechas_pub_min=pd.read_pickle(path_minutes_published)


df_fechas_pub_min.columns=['Meeting', 'Fecha Minuta']
df_fechas_pub_min['Meeting'] = pd.to_datetime(df_fechas_pub_min['Meeting'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_fechas_pub_min['Fecha Minuta'] = pd.to_datetime(df_fechas_pub_min['Fecha Minuta'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_fechas_pub_min.set_index('Meeting', inplace=True)
df_fechas_pub_min.index = pd.to_datetime(df_fechas_pub_min.index)

# Define the start and end dates for the data
start_date = datetime.datetime(1950, 1, 1)

# Fetch Federal Funds Rate data from FRED
ff_rate = web.DataReader([
                "DFF",
                'DFEDTAR',
                "DFEDTARL",
                "DFEDTARU"], 'fred', start_date)
ff_rate_mid = ff_rate[['DFEDTAR',
                "DFEDTARL",
                "DFEDTARU"]].mean(axis=1).ffill()
effective_ff_rate = ff_rate["DFF"].ffill().rolling(5).mean().dropna()
effective_ff_rate = ff_rate["DFF"].ffill()

other_short_rates1 = web.DataReader(["DTB4WK", 'DTB3',], 'fred', start_date).ffill()

path_1M_Rate='/content/1M Rate.csv'
un_mes_rate=pd.read_csv(path_1M_Rate)
un_mes_rate.set_index('Date', inplace=True)
un_mes_rate.index = pd.to_datetime(un_mes_rate.index)

# path_effective_yield_rate= '/content/drive/MyDrive/effective funds rate.csv'
# effective_ff_rate_history=pd.read_csv(path_effective_yield_rate)
# effective_ff_rate_history.set_index('Date', inplace=True)


In [ ]:
other_short_rates=pd.merge(other_short_rates1,un_mes_rate, right_index=True, left_index=True, how='left' )

In [ ]:
other_short_rates['Tasa 1 Mes_x'] = other_short_rates['DTB4WK'].combine_first(other_short_rates['Effective Yield'])


In [ ]:
other_short_rates['Tasa 1 Mes_x'].dropna()

DATE
1996-12-31    5.048
1997-01-02    5.026
1997-01-03    4.984
1997-01-06    4.982
1997-01-07    5.064
              ...  
2024-01-24    5.270
2024-01-25    5.280
2024-01-26    5.270
2024-01-29    5.270
2024-01-30    5.270
Name: Tasa 1 Mes_x, Length: 7051, dtype: float64

In [ ]:
other_short_rates.drop(columns=['DTB4WK', 'Effective Yield', ' ',], inplace=True)

In [ ]:
other_short_rates.columns=['DTB3', 'DTB4WK']
other_short_rates

,DTB3,DTB4WK
DATE,,
1954-01-04,1.33,NaN
1954-01-05,1.28,NaN
1954-01-06,1.28,NaN
1954-01-07,1.31,NaN
1954-01-08,1.31,NaN
...,...,...
2024-01-24,5.21,5.27
2024-01-25,5.21,5.28
2024-01-26,5.21,5.27


In [ ]:
#Chequear que el año 2018 este dentro del dataframe dataset_minutes
dataset_statements = MonetaryPolicyCommittee(historical_split = 2017).find_statements()
dataset_minutes = FederalReserveMins(historical_split = 2018).find_minutes()

Constructing links between 1994 and 2024
Extracting the past 219 FOMC Statements.
Retrieving articles.
...........................................................................................................................................................................................................................Constructing links between 1995 and 2024
Extracting Federal Reserve Minutes.
Retrieving articles.
..............................................................................................................................................................................................................................................

In [ ]:
fechas_pub_minutas = pd.merge(dataset_minutes, df_fechas_pub_min, left_index=True, right_index=True)
print(fechas_pub_minutas)

fechas = pd.DataFrame({
    "Fecha Reunión" : dataset_minutes.index[:],
    "Publicacion Minuta" : fechas_pub_minutas['Fecha Minuta'],})

fechas["Publicacion Minuta"] = pd.to_datetime(fechas["Publicacion Minuta"])
fechas["Fecha Reunión + 1 BD"] = fechas["Fecha Reunión"] + pd.offsets.BDay()
fechas["Publicacion Minuta + 1 BD"] = fechas["Publicacion Minuta"]+ pd.offsets.BDay()
fechas["Fecha Siguiente Reunión"] = list(dataset_minutes.index[1:])+[None]
fechas["Fecha Siguiente Reunión + 1 BD"] = fechas["Fecha Siguiente Reunión"]+ pd.offsets.BDay()

                                         Federal_Reserve_Mins Fecha Minuta
1995-02-01  The Federal Reserve, the central bank of the U...   1995-03-31
1995-03-28  A meeting of the Federal Open Market Committee...   1995-05-26
1995-05-23  A meeting of the Federal Open Market Committee...   1995-07-07
1995-07-06  A meeting of the Federal Open Market Committee...   1995-08-25
1995-08-22  A meeting of the Federal Open Market Committee...   1995-09-29
...                                                       ...          ...
2023-06-14  The Federal Reserve, the central bank of the U...   2023-07-05
2023-07-26  The Federal Reserve, the central bank of the U...   2023-08-16
2023-09-20  The Federal Reserve, the central bank of the U...   2023-10-11
2023-11-01  The Federal Reserve, the central bank of the U...   2023-11-21
2023-12-13  The Federal Reserve, the central bank of the U...   2024-01-03

[231 rows x 2 columns]


In [ ]:
tasas = pd.concat(
    (ff_rate_mid,
     effective_ff_rate,
    other_short_rates), axis=1
)
tasas.columns =["Fed Target Rate", "Effective Fed Funds Rate", "Tasa 3 Meses", "Tasa 1 Mes"]

df_fechas = fechas[['Fecha Reunión + 1 BD',
 'Publicacion Minuta + 1 BD',
 'Fecha Siguiente Reunión + 1 BD']]
variacion_tasas = df_fechas.merge(tasas, left_on='Fecha Siguiente Reunión + 1 BD', right_index=True, how="left")["Fed Target Rate"]-df_fechas.merge(tasas, left_on='Fecha Reunión + 1 BD', right_index=True, how="left")["Fed Target Rate"]
df_fechas["Decisión Política Monetaria"] = variacion_tasas
df_fechas = df_fechas.merge(tasas, left_on='Publicacion Minuta + 1 BD', right_index=True, how='left')

df_fechas.tail(10)


df_fechas['Tasa 1 Mes'].isna().sum(), df_fechas['Tasa 3 Meses'].isna().sum(),df_fechas['Effective Fed Funds Rate'].isna().sum()

<ipython-input-46-682ff1060aa9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fechas["Decisión Política Monetaria"] = variacion_tasas


(17, 0, 0)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_fechas.to_pickle("df_fechas.pkl")
tasas.to_pickle("tasas.pkl")
dataset_minutes.to_pickle("dataset_minutes.pkl")
dataset_statements.to_pickle("dataset_statements.pkl")


In [ ]:
files.download(r'df_fechas.pkl')
files.download(r'tasas.pkl')
files.download(r'dataset_minutes.pkl')
files.download(r'dataset_statements.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>